In [17]:
from IPython.display import clear_output
!wget https://disk.karelia.pro/djnSMfz/Трек%201.%20Онежское%20озеро.tgz?attachment
import os
try:
    os.mkdir("./data")
except: pass
!tar -xvzf "Трек 1. Онежское озеро.tgz?attachment" -C ./data
clear_output()

In [18]:
from google.colab import drive
drive.mount('/content/drive')
!pip install catboost

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# ваша/наша папка
folder = "Ситилинк Хакатон/Для организаторов/Ситилинк Хакатон/Рабочее решение"
# "./drive/My Drive/" обязательно должен быть
# по нему мы обращаемся к моделям
path = f"./drive/My Drive/{folder}/"
print(path)

./drive/My Drive/Ситилинк Хакатон/Для организаторов/Ситилинк Хакатон/Рабочее решение/


In [20]:
import os
os.listdir(path)

['2016-2021forecast.csv',
 'joined.csv',
 'info.ipynb',
 'joined_last.csv',
 'joined_new.csv',
 'random',
 'torch_train.ipynb',
 'baseline_v1.1.ipynb',
 'Models',
 'baseline_v1_2.ipynb',
 'ModernClassifier_train.ipynb',
 'ModernRegressor_train.ipynb']

In [21]:
import os
from torchvision import transforms
import pickle
import pandas as pd
import torch
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np
 
class LakeClassifier():
    def __init__(self, models_path):
        self.device = torch.device('cuda' if 
                                   torch.cuda.is_available() else 'cpu')
        self.network = torch.load(os.path.join(models_path, "network.pth"), map_location=torch.device('cpu')).to(self.device)
        self.transform = transforms.ToTensor()
        self.size = (128, 128)
    
    def predict_network(self, path):
        predictions = []
        for filename in tqdm(os.listdir(path)):
            with Image.open(os.path.join(path, filename)) as image:
                image = image.resize(self.size)
            X = self.transform(image)
            X = X.view(1, *X.size()).to(self.device)
            prediction = self.network(X).argmax(dim=1).item()
            predictions.append([filename, prediction])
        return predictions

In [22]:
model_nn = LakeClassifier(os.path.join(path, "Models"))
predictions = model_nn.predict_network("data")

In [23]:
import pandas as pd

# переводим дату в числа
def date_to_data(frame, holidays=None,
                 drop=False, name="date", suffix=""):
    assert name in frame, "Нужна колонка с датой"
    date = frame[name].dt

    info = pd.concat([date.days_in_month,
                     date.month,
                     date.quarter,
                     date.month % 12 // 3 + 1], axis=1)
    
    info.columns = ["days_in_month", "month",
                    "quarter", "season"]

    temp = pd.concat([date.isocalendar(),
                      info], axis=1)

    temp.columns = temp.columns + suffix
    frame = pd.concat([frame, temp], axis=1)
    if drop:
        return frame.drop([name], axis=1)
    return frame

df = pd.read_csv(path + "joined_last.csv", index_col="Unnamed: 0")
df["date"] = pd.to_datetime(df["name"].apply(lambda x: x[:-4]))
df = date_to_data(df, name="date", drop=False)

In [24]:
arr = []
frame = df[["date", "y"]].values
target = df["y"].to_list()

for i, data in enumerate(frame):
    date, y = data
    if y:
        # arr.append([date, y, 0])
        arr.append(0)
    else:
        idx = i + target[i:].index(1)
        temp_date, temp_y = frame[idx]
        new_date = temp_date - date
        new_date = (new_date.days * 24) + (new_date.seconds // 3600)
        if new_date < 0:
            print(temp_date, date)
            print(temp_date - date)
            print(i, idx)
            break
        # arr.append([date, y, new_date])
        arr.append(new_date)

df["hours_to_freezing"] = arr

In [25]:
predictions_df = pd.DataFrame(predictions, columns=["filename", "prediction"])
frame = df.merge(predictions_df, left_on="name", right_on="filename", how="left")

In [26]:
names = ['hour', 'year', 'week', 'day',
        'days_in_month', 'month', 'quarter', 'season',
        'prediction']

In [27]:
from sklearn.model_selection import train_test_split

def train_test_val_split(X, y,
                         test_size=0.3, val_size=0.3,
                         random_state=42, shuffle=True):
    """
    separator for train, test, validation
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        train_size=1 - (test_size + val_size),
                                                        random_state=random_state, shuffle=shuffle)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,
                                                    test_size=test_size / (test_size + val_size),
                                                    random_state=random_state, shuffle=shuffle)
    return X_train, y_train, X_test, y_test, X_val, y_val

X = frame[names].astype(np.float32)
y = frame["hours_to_freezing"].astype(np.int32)

X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(X, y,
                                                                      test_size=0.3,
                                                                      val_size=0.3,
                                                                      shuffle=False)

In [28]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=1000,
                          task_type="GPU",
                          loss_function="RMSE",
                          grow_policy='Depthwise')

# model.fit(X_train, y_train)
model.fit(X, y)

Learning rate set to 0.076706
0:	learn: 1983.7634680	total: 8.98ms	remaining: 8.97s
1:	learn: 1844.2136887	total: 15.8ms	remaining: 7.87s
2:	learn: 1718.3993660	total: 22.5ms	remaining: 7.49s
3:	learn: 1604.1419019	total: 29.4ms	remaining: 7.31s
4:	learn: 1507.3916044	total: 36.1ms	remaining: 7.18s
5:	learn: 1406.7328839	total: 42.8ms	remaining: 7.08s
6:	learn: 1322.1768565	total: 49.8ms	remaining: 7.07s
7:	learn: 1241.9344371	total: 56.4ms	remaining: 7s
8:	learn: 1162.8314414	total: 63.3ms	remaining: 6.97s
9:	learn: 1093.0162864	total: 69.6ms	remaining: 6.89s
10:	learn: 1025.9835647	total: 75.7ms	remaining: 6.81s
11:	learn: 967.8979487	total: 81.8ms	remaining: 6.73s
12:	learn: 912.2944718	total: 87.9ms	remaining: 6.68s
13:	learn: 854.9745202	total: 94.1ms	remaining: 6.63s
14:	learn: 809.9416244	total: 100ms	remaining: 6.58s
15:	learn: 761.5193235	total: 106ms	remaining: 6.54s
16:	learn: 718.3632926	total: 113ms	remaining: 6.51s
17:	learn: 676.9587699	total: 131ms	remaining: 7.13s
18:	

In [29]:
from sklearn.metrics import mean_absolute_error

prediction = pd.DataFrame(model.predict(X_test), columns=["pred"])
prediction[prediction["pred"] < 0] = 0
prediction = prediction["pred"].values

mean_absolute_error(y_test, prediction)

7.475298551184398

In [30]:
from sklearn.metrics import mean_absolute_error

prediction = pd.DataFrame(model.predict(X_val), columns=["pred"])
prediction[prediction["pred"] < 0] = 0
prediction = prediction["pred"].values

mean_absolute_error(y_val, prediction)

8.437558305545421

In [31]:
sorted(zip(model.feature_names_, model.feature_importances_),
       reverse=True, key=lambda x: x[1])

[('week', 47.015828487435556),
 ('season', 23.515697625991358),
 ('prediction', 13.657342580930601),
 ('year', 11.123500068570944),
 ('month', 3.0838171158750867),
 ('day', 0.8575589690691205),
 ('days_in_month', 0.5145212030583373),
 ('hour', 0.14875554560794066),
 ('quarter', 0.0829784034610598)]

In [32]:
import pickle
with open(path + "Models/ModernRegressor.pkl", "wb") as f:
    pickle.dump(model, f)